In [ ]:
# !pip install transformers

In [8]:
from transformers import DebertaTokenizer, DebertaModel
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import torch
from joblib import dump, load

In [2]:
train_data = pd.read_csv('train.csv')
val_data = pd.read_csv('dev.csv')

tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
model = DebertaModel.from_pretrained('microsoft/deberta-base')

In [3]:
def tokenize(premises, hypotheses, max_length=128):
    # Tokenize a list of premise and hypothesis pairs
    tokenized_output = tokenizer(premises, hypotheses,
                                 padding='max_length',
                                 truncation=True,
                                 max_length=max_length,
                                 return_tensors="pt",
                                 truncation_strategy='only_second')
    
    return tokenized_output

In [4]:
tokenized_train = tokenize([str(p) for p in train_data["premise"].tolist()], [str(h) for h in train_data["hypothesis"].tolist()])
tokenized_test = tokenize([str(p) for p in val_data["premise"].tolist()], [str(h) for h in val_data["hypothesis"].tolist()])

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [5]:
def get_embeddings(model, tokenized_input):
    # Move tokenized input to the same device as the model
    input_ids = tokenized_input['input_ids'].to(model.device)
    attention_mask = tokenized_input['attention_mask'].to(model.device)

    with torch.no_grad():
        # Forward pass, get model outputs
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # Get the embeddings from the last hidden state
        # You could opt for other strategies like taking the output of the [CLS] token
        embeddings = outputs.last_hidden_state.mean(dim=1)
    
    return embeddings.cpu().numpy()  # Assuming you want to continue in numpy, move tensor to CPU and convert to numpy

def get_embeddings_in_chunks(model, tokenized_inputs, chunk_size=128):
    # Split tokenized input into smaller chunks
    all_embeddings = []
    input_chunks = []

    for i in range(0, len(tokenized_inputs['input_ids']), chunk_size):
        input_chunks.append(tokenized_inputs[i:i + chunk_size])
    
    index = 1
    for chunk in input_chunks:
        print(f'{index}/{len(input_chunks)}')
        
        embeddings = get_embeddings(model, chunk) 
        all_embeddings.append(embeddings)
        
        index+=1

    # Concatenate all chunk embeddings
    return np.concatenate(all_embeddings, axis=0)

In [6]:
X_train = get_embeddings_in_chunks(model, tokenized_train)
X_test = get_embeddings_in_chunks(model, tokenized_test)

1/211
2/211
3/211
4/211
5/211
6/211
7/211
8/211
9/211
10/211
11/211
12/211
13/211
14/211
15/211
16/211
17/211
18/211
19/211
20/211
21/211
22/211
23/211
24/211
25/211
26/211
27/211
28/211
29/211
30/211
31/211
32/211
33/211
34/211
35/211
36/211
37/211
38/211
39/211
40/211
41/211
42/211
43/211
44/211
45/211
46/211
47/211
48/211
49/211
50/211
51/211
52/211
53/211
54/211
55/211
56/211
57/211
58/211
59/211
60/211
61/211
62/211
63/211
64/211
65/211
66/211
67/211
68/211
69/211
70/211
71/211
72/211
73/211
74/211
75/211
76/211
77/211
78/211
79/211
80/211
81/211
82/211
83/211
84/211
85/211
86/211
87/211
88/211
89/211
90/211
91/211
92/211
93/211
94/211
95/211
96/211
97/211
98/211
99/211
100/211
101/211
102/211
103/211
104/211
105/211
106/211
107/211
108/211
109/211
110/211
111/211
112/211
113/211
114/211
115/211
116/211
117/211
118/211
119/211
120/211
121/211
122/211
123/211
124/211
125/211
126/211
127/211
128/211
129/211
130/211
131/211
132/211
133/211
134/211
135/211
136/211
137/211
138/211
139/

In [11]:
svm = SVC(kernel='rbf', C=3.0) 
svm.fit(X_train, train_data['label'].tolist())

# Evaluate
y_pred = svm.predict(X_test)
print(classification_report(val_data['label'].tolist(), y_pred))

              precision    recall  f1-score   support

           0       0.65      0.67      0.66      3259
           1       0.68      0.66      0.67      3478

    accuracy                           0.67      6737
   macro avg       0.67      0.67      0.67      6737
weighted avg       0.67      0.67      0.67      6737



In [12]:
dump(svm, 'svm_DeBERTa.joblib')

['svm_DeBERTa.joblib']